# Our First LLMAgent — Hailstone

In [1]:
LOGGING_ENABLED = True

In [2]:
import logging

from llm_agents_from_scratch.logger import enable_console_logging

if LOGGING_ENABLED:
    enable_console_logging(logging.INFO)

## Define the Hailstone Tool

In [3]:
from pydantic import BaseModel

from llm_agents_from_scratch.tools import PydanticFunctionTool


class AlgoParams(BaseModel):
    """Params for next_number."""

    x: int


def next_number(params: AlgoParams) -> int:
    """Generate the next number of the sequence."""
    if params.x % 2 == 0:
        return params.x // 2
    return 3 * params.x + 1


# convert our Python function to a BaseTool
tool = PydanticFunctionTool(next_number)

## Define our backbone LLM

In [4]:
from llm_agents_from_scratch.llms import OllamaLLM

In [5]:
llm = OllamaLLM(model="qwen2.5:3b")

## Define the LLMAgent

In [6]:
from llm_agents_from_scratch import LLMAgent

In [7]:
llm_agent = LLMAgent(
    llm=llm,
    tools=[tool],
)

## Define the Hailstone Task

In [8]:
from llm_agents_from_scratch.data_structures import Task

In [9]:
instruction_template = """You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number x={x}.

<rules>
CALL `next_number` on the current number x
STOP AND WAIT for the result.
REPEAT this step-by-step process until the number 1 is reached.
FINAL RESULT: When you receive the number 1, provide the complete sequence
you observed from start to finish (including the starting number x and
ending number 1).
</rules>

<warnings>
NEVER fabricate or simulate tool call results
NEVER make multiple tool calls in one response
STOP and WAIT - ALWAYS wait for the actual tool response before deciding next steps
</warnings>
"""

In [10]:
task = Task(
    instruction=instruction_template.format(x=4),
)

In [11]:
handler = llm_agent.run(task)

INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number ...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the numb...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      🛠️ Executing Tool Call: next_number
INFO (llm_agents_fs.TaskHandler) :      ✅ Successful Tool Call: 2
INFO (llm_agents_fs.TaskHandler) :      ✅ Step Result: <tool_call>
{"name": "next_number", "arguments": {"x":2}}
</tool_call>
INFO (llm_agents_fs.TaskHandler) :      🧠 New Step: CALL `next_number` on the current number x=2
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: CALL `next_number` on the current number x=2
INFO (llm_agents_fs.TaskHandler) :      🛠️ Executing Tool Call: next_number
INFO (llm_agents_fs.TaskHandler) :      ✅ Succe

In [12]:
handler.done()

True

# See the TaskResult

In [13]:
result = handler.result()

In [14]:
print(result)

The complete sequence observed from start to finish (including the starting number x and ending number 1) is: 4, 2, 1.


### See the Rollout

In [15]:
print(handler.rollout)

=== Task Step Start ===

💬 assistant: The current instruction is 'You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number x=4.

<rules>
CALL `next_number` on the current number x
STOP AND WAIT for the result.
REPEAT this step-by-step process until the number 1 is reached.
FINAL RESULT: When you receive the number 1, provide the complete sequence
you observed from start to finish (including the starting number x and
ending number 1).
</rules>

<warnings>
NEVER fabricate or simulate tool call results
NEVER make multiple tool calls in one response
STOP and WAIT - ALWAYS wait for the actual tool response before deciding next steps
</warnings>
'

💬 assistant: I need to make the following tool call(s):

{
    "tool_name": "next_number",
    "arguments": {
        "x": 4
    }
}.

💬 tool: {
    "tool_call": {
        "tool_name": "next_number",
        "arguments": {
            "x": 4
        }
    },
    "content":

In [16]:
# number of task steps executed
handler.step_counter

2

## Evaluation

In [30]:
from dataclasses import dataclass


@dataclass
class Example:
    """Benchmark example."""

    number: int
    sequence: list[int]

    @property
    def min_steps(self) -> int:
        """Returns the minimum number of TaskSteps to perform the Task."""
        return len(self.sequence) - 1

In [31]:
benchmark: list[Example] = [
    Example(
        number=11,
        sequence=[11, 34, 17, 52, 26, 13, 40, 20, 10, 5, 16, 8, 4, 2, 1],
    ),
    Example(number=7, sequence=[4, 2, 1]),
    Example(number=4, sequence=[4, 2, 1]),
    Example(number=3, sequence=[3, 10, 5, 16, 8, 4, 2, 1]),
    Example(number=2, sequence=[2, 1]),
]

### Run tasks

In [34]:
handlers = []
for ex in benchmark:
    h = llm_agent.run(
        Task(
            instruction=instruction_template.format(x=ex.number),
            max_steps=20,
        ),
    )
    handlers.append(h)

INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number ...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the numb...[TRUNCATED]
INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number ...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the numb...[TRUNCATED]
INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number

In [62]:
[h.done() for h in handlers]

[True, True, True, True, True]

In [64]:
[h.result() for h in handlers]

[TaskResult(task_id='6225e86d-837f-43a3-b58a-753e4f78f975', content='[11, 34, 17, 52, 26, 13, 40, 20, 10, 5, 16, 8, 4, 2, 1]'),
 TaskResult(task_id='4d505a87-274e-4195-b346-dc3c55f51854', content='[7, 22]'),
 TaskResult(task_id='e7a2186c-a6bd-4be4-bb11-cd1e9ff18792', content='4 → 2 → 1'),
 TaskResult(task_id='73215131-4645-49e0-ae68-9b3dd265cf5c', content='\n3 → 10 → 5 → 16 → 8 → 4 → 2 → 1\n'),
 TaskResult(task_id='4a878b13-b074-40b7-9a55-6b9e356bca0a', content='[2, 4, 2, 1]')]

### Setup Judge LLM

In [65]:
judge_prompt_template = """You are an evaluation assistant. Given a number and its correct
sequence, use them to assess whether another assistants final result contains the correct sequence.

<number>
{number}
</number>

<sequence>
{sequence}
</sequence>

<result>
{result}
</result>
"""

#### Structured outputs for final result correctness evaluation

In [66]:
from pydantic import Field


class ExampleResultEvaluation(BaseModel):
    """Evaluation of result."""

    correct: bool = Field(
        description="True if the assistant's final result contains the correct sequence. False otherwise.",
    )

In [73]:
import asyncio

eval_async_tasks = []
for ex, handler in zip(benchmark, handlers, strict=False):
    async_task = llm.structured_output(
        prompt=judge_prompt_template.format(
            number=ex.number,
            sequence=ex.sequence,
            result=str(handler.result()),
        ),
        mdl=ExampleResultEvaluation,
    )
    eval_async_tasks.append(async_task)

result_evals = await asyncio.gather(*eval_async_tasks)
result_evals

[ExampleResultEvaluation(correct=True),
 ExampleResultEvaluation(correct=False),
 ExampleResultEvaluation(correct=True),
 ExampleResultEvaluation(correct=False),
 ExampleResultEvaluation(correct=False)]

#### Extra steps evaluation

In [74]:
extra_steps = [
    abs(h.step_counter - ex.min_steps)
    for h, ex in zip(handlers, benchmark, strict=False)
]
extra_steps

[0, 1, 0, 11, 3]

### Create Eval summary report

In [80]:
!uv pip install pandas tabulate -q

In [81]:
import pandas as pd

In [95]:
report = pd.DataFrame(
    data={
        "result_correctness": [int(el.correct) for el in result_evals],
        "number_extra_steps": extra_steps,
    },
)
report.loc["Average"] = report.mean().round(2)
print(report.to_markdown())

|         |   result_correctness |   number_extra_steps |
|:--------|---------------------:|---------------------:|
| 0       |                  1   |                    0 |
| 1       |                  0   |                    1 |
| 2       |                  1   |                    0 |
| 3       |                  0   |                   11 |
| 4       |                  0   |                    3 |
| Average |                  0.4 |                    3 |
